# 📸 ImaLink API Testing Notebook

**Rask testing av import-funksjonalitet direkte i Jupyter**

Dette er en interaktiv notebook for å teste ImaLink API-en uten å måtte redigere HTML-filer.

In [ ]:
# Setup
import requests
import json
import time
from datetime import datetime
from pathlib import Path

# Konfigurasjon
API_BASE = "http://localhost:8000/api/v1"
TEST_SOURCE = "C:/temp/PHOTOS_SRC_TEST_MICRO"

print("🚀 ImaLink API Tester Ready!")
print(f"API Base: {API_BASE}")
print(f"Test Source: {TEST_SOURCE}")

## 🔗 Test Connection

In [ ]:
# Test API connection
def test_connection():
    try:
        response = requests.get(f"{API_BASE}/import_sessions/test")
        if response.ok:
            print("✅ API Connection OK!")
            return response.json()
        else:
            print(f"❌ Connection failed: {response.status_code}")
            return None
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

test_connection()

## 📁 Storage Info

In [ ]:
# Get storage information
def get_storage_info(subfolder=None):
    try:
        url = f"{API_BASE}/import_sessions/storage-info"
        if subfolder:
            url += f"?subfolder={subfolder}"
        
        response = requests.get(url)
        if response.ok:
            print("📁 Storage Info:")
            data = response.json()
            for key, value in data.items():
                print(f"  {key}: {value}")
            return data
        else:
            print(f"❌ Failed: {response.status_code}")
            return None
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

get_storage_info("notebook_test")

## 🚀 Start Import (Unified with File Copy)

In [ ]:
# Start unified import (database + file copy)
def start_import(source_dir, description="Notebook Test", storage_subfolder=None, copy_files=True):
    payload = {
        "source_directory": source_dir,
        "source_description": description,
        "copy_files": copy_files
    }
    
    if storage_subfolder:
        payload["storage_subfolder"] = storage_subfolder
    
    try:
        print(f"🚀 Starting import from: {source_dir}")
        print(f"📝 Description: {description}")
        print(f"📁 Copy files: {copy_files}")
        
        response = requests.post(
            f"{API_BASE}/import_sessions/",
            json=payload
        )
        
        if response.ok:
            result = response.json()
            session_id = result["import_id"]
            print(f"✅ Import started! Session ID: {session_id}")
            print(f"📊 Status: {result.get('status')}")
            return session_id
        else:
            print(f"❌ Import failed: {response.status_code}")
            print(response.text)
            return None
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

# Start a test import
session_id = start_import(
    source_dir=TEST_SOURCE,
    description=f"Notebook Test - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    storage_subfolder="notebook_test",
    copy_files=True
)

print(f"\n💾 Session ID stored: {session_id}")

## 📊 Monitor Progress

In [ ]:
# Monitor import progress
def get_import_status(session_id):
    try:
        response = requests.get(f"{API_BASE}/import_sessions/status/{session_id}")
        
        if response.ok:
            data = response.json()
            
            print(f"📊 Status for Session {session_id}:")
            print(f"  Status: {data.get('status')}")
            print(f"  Progress: {data.get('progress_percentage', 0)}%")
            print(f"  Files Found: {data.get('total_files_found', 0)}")
            print(f"  Images Imported: {data.get('images_imported', 0)}")
            print(f"  Duplicates Skipped: {data.get('duplicates_skipped', 0)}")
            print(f"  Files Copied: {data.get('files_copied', 0)}")
            print(f"  Storage Path: {data.get('storage_path', 'N/A')}")
            
            return data
        else:
            print(f"❌ Failed to get status: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

# Check status if we have a session ID
if session_id:
    status = get_import_status(session_id)
else:
    print("⚠️  No session ID available. Run the import cell first.")

## 🔄 Auto-Monitor (Run and wait for completion)

In [ ]:
# Auto-monitor until completion
def monitor_until_complete(session_id, max_wait=300):
    if not session_id:
        print("⚠️  No session ID provided")
        return None
    
    print(f"🔄 Monitoring session {session_id} until completion...")
    start_time = time.time()
    
    while time.time() - start_time < max_wait:
        status_data = get_import_status(session_id)
        
        if not status_data:
            break
            
        status = status_data.get('status')
        progress = status_data.get('progress_percentage', 0)
        
        print(f"⏱️  {datetime.now().strftime('%H:%M:%S')} - Status: {status}, Progress: {progress}%")
        
        if status in ['completed', 'failed']:
            print(f"\n🎉 Import {status}!")
            return status_data
            
        time.sleep(5)
    
    print(f"⏰ Timeout after {max_wait} seconds")
    return get_import_status(session_id)

# Monitor if we have a session
if session_id:
    final_status = monitor_until_complete(session_id)
    if final_status:
        print("\n📋 Final Results:")
        print(json.dumps(final_status, indent=2))
else:
    print("⚠️  No session ID available. Run the import cell first.")

## 📋 List All Imports

In [ ]:
# List all import sessions
def list_all_imports():
    try:
        response = requests.get(f"{API_BASE}/import_sessions/")
        
        if response.ok:
            data = response.json()
            imports = data.get('imports', [])
            
            print(f"📋 Found {len(imports)} import sessions:")
            
            for imp in imports[-10:]:  # Show last 10
                print(f"\n  ID: {imp.get('id')}")
                print(f"    Status: {imp.get('status')}")
                print(f"    Source: {imp.get('source_path')}")
                print(f"    Images: {imp.get('images_imported', 0)}")
                print(f"    Files Copied: {imp.get('files_copied', 0)}")
                print(f"    Started: {imp.get('started_at')}")
            
            return imports
        else:
            print(f"❌ Failed: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

list_all_imports()

## 🔧 Quick Test Functions

In [ ]:
# Quick one-liner functions for rapid testing

def quick_test():
    """Run a complete test cycle"""
    print("🎯 Quick Test Cycle")
    
    # 1. Test connection
    if not test_connection():
        return
    
    # 2. Get storage info
    get_storage_info("quick_test")
    
    # 3. Start import
    sid = start_import(
        TEST_SOURCE, 
        "Quick Test", 
        "quick_test"
    )
    
    if sid:
        print(f"\n✅ Started session {sid}")
        print("Monitor manually or run monitor_until_complete() function")
    
    return sid

def test_custom_path(source_path):
    """Test with custom source path"""
    return start_import(
        source_path,
        f"Custom Test - {Path(source_path).name}",
        f"custom_{datetime.now().strftime('%H%M%S')}"
    )

# Run quick test
print("Ready for testing! 🚀")
print("\nQuick commands:")
print("- quick_test() - Full test cycle")
print("- test_custom_path('C:/your/path') - Test custom directory")
print("- get_import_status(session_id) - Check specific session")
print("- monitor_until_complete(session_id) - Wait for completion")